In [12]:
import random
import pandas as pd
import os

from tqdm import tqdm

# Requires the XOR-AttribQA dataset from: https://storage.googleapis.com/gresearch/xor_attriqa/xor_attriqa.zip (https://github.com/google-research/google-research/tree/master/xor_attriqa)

# TODO might be a way to get correct answers even if not ais

random.seed(42)

SPLIT = "test" # Choose between "dev" and "test"

if SPLIT == "test":
    DATASET_PATH = "/home/wesley/dev/ARES/multilingual_data/xor_attriqa/in-language/bn.jsonl"
elif SPLIT == "dev":
    DATASET_PATH = "/home/wesley/dev/ARES/multilingual_data/xor_attriqa/in-language/val.jsonl"

print(DATASET_PATH)

xor_attriqa_split = pd.read_json(DATASET_PATH, lines=True)

if SPLIT == "dev":
    xor_attriqa_split = xor_attriqa_split[xor_attriqa_split["query_language"] == "bn"]

# Named to be consistent with the mlqa code
dataset_merged = pd.DataFrame()
dataset_merged["Document_en"] = xor_attriqa_split["passage_en"]
dataset_merged["Answer_en"] = xor_attriqa_split["prediction_translated_en"]
dataset_merged["Query_en"] = xor_attriqa_split["query_translated_en"]
dataset_merged["Document_bn"] = xor_attriqa_split["passage_in_language"]
dataset_merged["Answer_bn"] = xor_attriqa_split["prediction"]
dataset_merged["Query_bn"] = xor_attriqa_split["query"]
dataset_merged["Answer_Faithfulness_Label"] = xor_attriqa_split["ais"].apply(lambda x: 1 if x == True else 0)
dataset_merged["id"] = dataset_merged["Document_en"].astype(str) + dataset_merged["Query_en"].astype(str)
dataset_merged["id"] = dataset_merged["id"].apply(hash)

/home/wesley/dev/ARES/multilingual_data/xor_attriqa/in-language/bn.jsonl


In [13]:
dataset_merged

,Document_en,Answer_en,Query_en,Document_bn,Answer_bn,Query_bn,Answer_Faithfulness_Label,id
0,"Atul Prasad Sen (October 20, 1871 – August 26,...","October 20, 1871",When was Atul Prasad Sen born? [b],"অতুলপ্রসাদ সেন (২০শে অক্টোবর, ১৮৭১- ২৬শে আগস্ট...","২০ অক্টোবর, ১৮৭১",অতুলপ্রসাদ সেনের জন্ম কবে হয় ?,1,-9195194734555365822
1,"Professor Dr. Yazuddin Ahmed (February 1, 1931...",29 October 2002,Professor Dr. When was Yazuddin Ahmed appointe...,অধ্যাপক ড. ইয়াজউদ্দিন আহম্মেদ (জন্ম: ফেব্রুয়...,২০০২ সালের ২৯ অক্টোবর,অধ্যাপক ড. ইয়াজউদ্দিন আহম্মেদ কবে বাংলাদেশের ...,0,2453882502232412446
2,Rao said he has no objection if Hyderabad is t...,Kirankumar Reddy,Who is the current Chief Minister of Andhra Pr...,"রাও বলেন, ১০ বছর হায়দ্রাবাদ উভয় রাজ্যের রাজধ...",কিরণকুমার রেড্ডি,অন্ধ্রপ্রদেশের বর্তমান মুখ্যমন্ত্রী কে ?,0,-5804486675830190615
3,"Vidya Balan (; born January 1, 1978) is an Ind...",1 January 1978,When was actress Vidya Balan born? [b],"বিদ্যা বালান (; জন্ম জানুয়ারি ১, ১৯৭৮) একজন ভ...",১ জানুয়ারি ১৯৭৮,অভিনেত্রী বিদ্যা বালান কবে জন্মগ্রহণ করেন ?,1,-2537750976005375488
4,"Sir Donald George Bradman, (; 27 August 1908 –...",28067,Australian cricketer Don Bradman's total numbe...,"স্যার ডোনাল্ড জর্জ ব্র্যাডম্যান, (; ২৭ আগস্ট ১...",২৮০৬৭,অস্ট্রেলিয়ান ক্রিকেটার ডন ব্র্যাডম্যানের আন্ত...,1,4512291728554537609
...,...,...,...,...,...,...,...,...
1402,"Dr. Ali Muhammad al-Sallabi, or ""al-Salabi"" (;...",1935,In what year did Hussain bin Ali declare himse...,"ডক্টর আলী মুহাম্মদ আল-সাল্লাবি, বা ""আল-সালাবি""...",১৯৩৫,হুসাইন বিন আলী কত সালে নিজেকে মক্কার বাদশা ব'ল...,0,-827006801374723934
1403,3 November 1975 Bangladesh coup d'état was a m...,Syed Mujahid Rahman,Who was the president of Bangladesh in 1971? [b],3 নভেম্বর 1975 বাংলাদেশ অভ্যুত্থান একটি সামরিক...,সৈয়দ মুজাহিদ Rahman,১৯৭১ সালে বাংলাদেশের রাষ্ট্রপতি কে ছিলেন ?,0,7811970130894867745
1404,"Sheikh Hasina Wazed (; , ; born 28 September 1...",Sheikh Hasina Wazed,Who was the finance minister of Bangladesh in ...,"শেখ হাসিনা ওয়াজেদ (; , ; জন্ম ২৮ সেপ্টেম্বর 1...",শেখ হাসিনা ওয়াজেদ,২০১১ সালে বাংলাদেশের অর্থমন্ত্রী কে ছিলেন ?,0,2569785700485853003
1405,in which they managed three wins and finished ...,Michael Clarke,Who was the vice-captain of the Indian team du...,যেখানে তারা তিনটি জয় পেয়েছে এবং 18 পয়েন্ট ন...,মাইকেল ক্লার্ক,২০১৫ ক্রিকেট বিশ্বকাপের সময় ভারতীয় দলের সহ-অধি...,0,1354123234652655000
